# **Import Libraries**


In [34]:
import numpy as np
import pandas as pd

import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support as score
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras import layers
from keras.models import Model

# Download Dataset to Colab

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d mengcius/cinic10
! unzip cinic10.zip

# Get Train, Validation and Test Data

In [28]:
def get_cinic_data(train_path, valid_path, test_path, image_generator):

  train_data = image_generator.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode="sparse",
    batch_size=64,
    shuffle=False,
  )

  validation_data = image_generator.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    class_mode="sparse",
    batch_size=64,
    shuffle=False,
  )

  test_data = image_generator.flow_from_directory(
    test_path,
    target_size=(224, 224),
    class_mode="sparse",
    batch_size=64,
    shuffle=False,
  )
  
  return train_data, validation_data, test_data

In [5]:
train_path = 'train'
valid_path = 'valid'
test_path = 'test'

In [26]:
image_generator = ImageDataGenerator(rescale=1./255)

In [29]:
# If the model is going to be trained with VGG or Resnet Models
image_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_data, valid_data, test_data = get_cinic_data(train_path, 
                                                        valid_path, 
                                                        test_path, 
                                                        image_generator)

### Get Labels For Confussion Matrix


In [12]:
IMAGE_COUNT = 9000
CLASS_COUNT =10

In [13]:
# Original labels of images
def get_labels():
  
  y_true = []
  for i in range(CLASS_COUNT):
    for j in range(IMAGE_COUNT):
      y_true.append(i)
  return y_true

In [14]:
y_true = get_labels()

# Functions

In [15]:
def get_confussion_matrix(predictions):
  
  y_pred = np.argmax(predictions, axis=1)

  confussion_matrix = np.zeros((10,10))
  k=-1
  for i in range(len(y_pred)):
    if i%IMAGE_COUNT == 0:
      k+=1
    confussion_matrix[k][y_pred[i]]+=1

  df = pd.DataFrame(data=confussion_matrix.T, 
                    index=["airplane", "automobile", "bird", "cat", "deer", 
                           "dog", "frog", "horse", "ship", "truck"],
                  columns=["airplane", "automobile", "bird", "cat", "deer", 
                           "dog", "frog", "horse", "ship", "truck"])
  return df

In [22]:
def build_model(filter_size=32, kernel=3,dropout_number=0.2):
  
  model = Sequential()

  #First Layer
  model.add(layers.Conv2D(filter_size, 
                          kernel, 
                          activation="relu",
                          kernel_initializer='glorot_uniform',
                          input_shape=(224,224,3)))
  
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Dropout(dropout_number))

  #Second Layer
  model.add(layers.Conv2D(filter_size, 
                          kernel, 
                          kernel_initializer='glorot_uniform', 
                          activation="relu"))
  
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Dropout(dropout_number))

  #Third Layer
  model.add(layers.Conv2D(filter_size, 
                          kernel, 
                          kernel_initializer='glorot_uniform', 
                          activation="relu"))
  
  model.add(layers.MaxPooling2D(2))
  model.add(layers.Dropout(dropout_number))

  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation="relu"))
  model.add(layers.Dense(10, activation="softmax"))

  model.compile(loss="sparse_categorical_crossentropy", 
                optimizer="adam", 
                metrics=["accuracy"])

  return model

In [17]:
# To get f1, precission and recall scores
def get_class_results(y_true, predictions):
  
  y_pred = np.argmax(predictions, axis=1)

  precision, recall, f1, _ = score(y_true, y_pred)

  class_df = pd.DataFrame(data=[precision, recall, f1], 
                          index=["precision", "recall", "f1"],
                          columns=["airplane", "automobile", "bird", "cat", 
                                   "deer", "dog", "frog", 
                                   "horse", "ship", "truck"])
  return class_df

# **Build Convolutional Neural Network Model**

In [ ]:
model = build_model()
model.summary()

In [ ]:
history = model.fit(train_data, batch_size=64, 
                    epochs=10, validation_data=valid_data)

In [ ]:
result = model.evaluate(test_data)

In [ ]:
predictions = model.predict(test_data)

confussion_df = get_confussion_matrix(predictions)

f1_df = get_class_results(y_true, predictions)

In [ ]:
# Prints Confussion Matrix
confussion_df

In [ ]:
# Prints Precision Recall and F1 Score 
f1_df

# Transfer Learning Functions

In [33]:
def create_trainable_model(pretrained_model, trainable_layers):
  
  for layer in pretrained_model.layers[:-trainable_layers]:
    layer.trainable = False

  pretrained_model._layers.pop()
  new_layer = layers.Dense(1024, activation='relu')(pretrained_model.layers[-1].output)
  output = layers.Dense(10, activation='softmax')(new_layer)
  model = Model(inputs=pretrained_model.inputs, outputs=output)
  print(model.summary)
  
  return model

# **VGG-16**

In [ ]:
from keras.applications.vgg16 import VGG16

Vgg_model = VGG16()
Vgg_model.summary()

In [ ]:
new_vgg_model = create_trainable_model(Vgg_model, 4)
new_vgg_model.summary()

In [ ]:
vgg_history = new_vgg_model.fit(train_data, batch_size=64, 
                                epochs=10, validation_data=valid_data)

In [ ]:
vgg_test = new_vgg_model.evaluate(test_data)

In [ ]:
predictions =  new_vgg_model.predict(test_data)

confussion_df = get_confussion_matrix(predictions)

f1_df = get_class_results(y_true, predictions)

In [ ]:
confussion_df

In [ ]:
f1_df

# **Resnet-50**

In [ ]:
from keras.applications.resnet50 import ResNet50
# load model
resnet_model = ResNet50(include_top=True,weights='imagenet')

In [ ]:
new_resnet_model = create_trainable_model(resnet_model, 4)
new_resnet_model.summary()

In [ ]:
history = model.fit(train_data, batch_size=64, 
                    epochs=10, validation_data=valid_data)

In [ ]:
results = model.evaluate(test_data)

In [ ]:
predictions = model.predict(test_data)

confussion_df = get_confussion_matrix(predictions)
f1_df = get_class_results(y_true, predictions)

In [ ]:
confussion_df

In [ ]:
f1_df